In [1]:
import pandas as pd
import numpy as np
import spacy
import re
from spacy.tokenizer import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [2]:
df = pd.read_csv('train.csv')
df


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [5]:
df['keyword'].unique()


array([nan, 'ablaze', 'accident', 'aftershock', 'airplane%20accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
       'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
       'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard', 'blood',
       'bloody', 'blown%20up', 'body%20bag', 'body%20bagging',
       'body%20bags', 'bomb', 'bombed', 'bombing', 'bridge%20collapse',
       'buildings%20burning', 'buildings%20on%20fire', 'burned',
       'burning', 'burning%20buildings', 'bush%20fires', 'casualties',
       'casualty', 'catastrophe', 'catastrophic', 'chemical%20emergency',
       'cliff%20fall', 'collapse', 'collapsed', 'collide', 'collided',
       'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew',
       'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris',
       'deluge', 'deluged', 'demolish', 'demolished', 'demolition',
       'derail', 'der

In [6]:
df['keyword'].value_counts()

fatalities               45
deluge                   42
armageddon               42
body%20bags              41
harm                     41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64

In [7]:
df['location'].unique()

array([nan, 'Birmingham', 'Est. September 2012 - Bristol', ...,
       'Vancouver, Canada', 'London ', 'Lincoln'], dtype=object)

In [8]:
df['location'].value_counts()

USA                      104
New York                  71
United States             50
London                    45
Canada                    29
                        ... 
Manavadar, Gujarat         1
Leicester, England         1
Augusta, Maine, 04330      1
PARACHUTE                  1
taco bell                  1
Name: location, Length: 3341, dtype: int64

In [9]:
df.loc[1]

id                                               4
keyword                                        NaN
location                                       NaN
text        Forest fire near La Ronge Sask. Canada
target                                           1
Name: 1, dtype: object

In [10]:
nlp = spacy.load("en_core_web_sm")

count = 0

df['label'] = df['location']

for i in df['text']:
    
    doc = nlp(i)
    for j in doc.ents:
        if (j.label_ == 'GPE') and (pd.isnull(df['location'][count])): 
            #print(df.loc[count])
            #print("Text:", j.text)
            #print("Label", j.label_)
            #print(df['location'][count])
            
            df['label'][count] = j.text
    count += 1

<ipython-input-10-557d5165a0fe>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][count] = j.text


In [11]:
def fill_location(df_x):
    
    
    nlp = spacy.load("en_core_web_sm")
    
    count = 0
    
    doc = nlp(df_x['text'])
    try:
        if doc.ents[0].label_ == 'GPE':
            #print(" printing label text", doc.ents[0].text)
            df_x['label'][count]==doc.ents[0].text
            count+=1
    except:
            count+=1
   
    return df_x['label']
    
#df['label'] = df['location']
#
#df['label'] = df.apply(lambda x: fill_location(x), axis=1)


#print(df_new['label'])

In [12]:
df['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [13]:
df['label'].value_counts()

USA                      105
New York                  72
United States             50
London                    48
California                32
                        ... 
Leicester, England         1
Augusta, Maine, 04330      1
PARACHUTE                  1
Alberta, VA                1
taco bell                  1
Name: label, Length: 3517, dtype: int64

In [14]:
nlp = spacy.load("en_core_web_sm")

print(nlp.Defaults.stop_words)


{'‘s', 'very', 'at', 'twenty', 'of', 'themselves', 'beyond', 'often', 'put', 'get', 'somewhere', 'our', 'other', 'please', 'yourself', 'is', 'becoming', 'whether', 'whereafter', 'they', 'only', 'somehow', 'among', 'throughout', 'anything', 'sometimes', 'been', 'their', 'none', 'afterwards', 'yourselves', 'side', '’d', 'latterly', 'rather', 'could', 'someone', 'since', 'back', 'regarding', 'just', 'this', 'that', 'unless', 'do', 'no', 'which', 'until', 'done', 'move', 'much', '‘d', 'hers', 'never', 'nine', 'thereupon', 'under', 'former', 'elsewhere', 'next', 'can', 'front', 'herself', 'us', 'what', 'hereupon', 'a', 'bottom', 'perhaps', 'nor', 'above', 'enough', 'upon', 'whole', 'take', 'see', 'thereby', 'name', 'too', 'nobody', 'nevertheless', "'d", 'meanwhile', 'neither', 'without', 'be', '‘m', 'up', 'herein', 'own', 'wherein', 'toward', 'well', 'wherever', 'still', 'quite', 'together', 'amongst', 'him', 'where', "'ve", '’ve', 'mostly', 'by', 'are', 'as', 'cannot', 're', 'go', 'if', 'f

In [15]:
stop_words = nlp.Defaults.stop_words

count = 0


df['text_no_stop'] = df['text']

for text in df['text']:
    words = []
    for word in text.split():
        if word.lower() not in stop_words:
            words.append(word)
            
    df['text_no_stop'][count] = re.sub(r'[^0-9a-zA-Z_//.]+'," ",str(words))
    count+=1
    

<ipython-input-15-c280988d88ea>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_no_stop'][count] = re.sub(r'[^0-9a-zA-Z_//.]+'," ",str(words))


In [38]:
train_data_size = round(len(df)*.665)
train_data = df.iloc[:train_data_size]
val_data = df.iloc[train_data_size:]

In [39]:
df

,id,keyword,location,text,target,label,text_no_stop
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,NaN,Deeds Reason earthquake ALLAH Forgive
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Canada,Forest fire near La Ronge Sask. Canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,NaN,residents asked shelter place notified office...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,California,13 000 people receive wildfires evacuation or...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,NaN,got sent photo Ruby Alaska smoke wildfires po...
...,...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,NaN,giant cranes holding bridge collapse nearby h...
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,California,aria_ahrary TheTawniest control wild fires Ca...
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,NaN,M1.94 01 04 UTC 5km S Volcano Hawaii. http //...
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,NaN,Police investigating e bike collided car Litt...


In [40]:
x = df['text'][3]
print(x)

y = re.sub(r'[^(,\d+)0-9,a-zA-Z_//.]+'," ",str(x))
print(y)

13,000 people receive #wildfires evacuation orders in California 
13,000 people receive wildfires evacuation orders in California 


In [41]:
# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

In [42]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
        lowercase = tf.strings.lower(input_data)
        stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
        return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')

In [43]:
input = tf.convert_to_tensor(train_data['text'])
target = tf.convert_to_tensor(train_data['target'])
input_val = tf.convert_to_tensor(val_data['text'])
target_val = tf.convert_to_tensor(val_data['target'])

In [45]:
print(input)
print(target)
print(input_val)
print(target_val)

tf.Tensor(
[b'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'
 b'Forest fire near La Ronge Sask. Canada'
 b"All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected"
 ...
 b'Rise up like a natural disaster we take the bat and then we take back the town????'
 b'its only getting colder and colder and faster and faster and when i first realized it it was like a natural disaster'
 b'anyway 2 me? Mateo just doesnt exist? Hes a mirage a pointless addition to our Generation. a human natural disaster. Im sorry but its true'], shape=(5063,), dtype=string)
tf.Tensor([1 1 1 ... 0 1 0], shape=(5063,), dtype=int64)
tf.Tensor(
[b'natural disaster \xc2\x89\xc3\x9b\xc3\x92 News Stories About natural disaster - Page 1 | Newser http://t.co/TB8gZEMbXU'
 b"And you're loving me like water slipping through my fingers such a natural disaster love"
 b'I added a video to a @YouTube playlist http://t.co/v2yXurne2

In [46]:
batch_size = 4
train_ds = tf.data.Dataset.from_tensor_slices((input,target))
train_ds = train_ds.batch(batch_size)

val_ds = tf.data.Dataset.from_tensor_slices((input_val,target_val))
val_ds = val_ds.batch(batch_size)

In [47]:

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    #standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)


# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
vectorize_layer.adapt(input)

In [48]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

In [26]:
with tf.compat.v1.Session() as sess:
    input = tf.convert_to_tensor(df['text'])
    output = tf.convert_to_tensor(df['target'])
    print(input.eval())
    print(output.eval())

[b'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'
 b'Forest fire near La Ronge Sask. Canada'
 b"All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected"
 ... b'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ'
 b'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.'
 b'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d']
[1 1 1 ... 1 1 1]


In [49]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [50]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [51]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
1266/1266 [==============================] - 4s 3ms/step - loss: 0.6829 - accuracy: 0.5819 - val_loss: 0.6816 - val_accuracy: 0.5475
Epoch 2/15
1266/1266 [==============================] - 4s 3ms/step - loss: 0.6506 - accuracy: 0.5831 - val_loss: 0.6371 - val_accuracy: 0.5475
Epoch 3/15
1266/1266 [==============================] - 4s 3ms/step - loss: 0.5328 - accuracy: 0.7061 - val_loss: 0.5610 - val_accuracy: 0.6475
Epoch 4/15
1266/1266 [==============================] - 3s 3ms/step - loss: 0.4087 - accuracy: 0.8151 - val_loss: 0.5170 - val_accuracy: 0.7282
Epoch 5/15
1266/1266 [==============================] - 3s 3ms/step - loss: 0.3298 - accuracy: 0.8552 - val_loss: 0.5015 - val_accuracy: 0.7498
Epoch 6/15
1266/1266 [==============================] - 4s 3ms/step - loss: 0.2711 - accuracy: 0.8845 - val_loss: 0.5066 - val_accuracy: 0.7584
Epoch 7/15
1266/1266 [==============================] - 4s 3ms/step - loss: 0.2254 - accuracy: 0.9056 - val_loss: 0.5262 - val_accuracy:

In [52]:
df_test = pd.read_csv('test.csv')
print(df_test)

         id keyword location  \
0         0     NaN      NaN   
1         2     NaN      NaN   
2         3     NaN      NaN   
3         9     NaN      NaN   
4        11     NaN      NaN   
...     ...     ...      ...   
3258  10861     NaN      NaN   
3259  10865     NaN      NaN   
3260  10868     NaN      NaN   
3261  10874     NaN      NaN   
3262  10875     NaN      NaN   

                                                   text  
0                    Just happened a terrible car crash  
1     Heard about #earthquake is different cities, s...  
2     there is a forest fire at spot pond, geese are...  
3              Apocalypse lighting. #Spokane #wildfires  
4         Typhoon Soudelor kills 28 in China and Taiwan  
...                                                 ...  
3258  EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...  
3259  Storm in RI worse than last hurricane. My city...  
3260  Green Line derailment in Chicago http://t.co/U...  
3261  MEG issues Hazardous Weather 

In [69]:
batch_size = 4


input_test = tf.convert_to_tensor(df_test['text'])

predict = model.predict(input_test)

print(predict )

#predict = scaler.inverse_transform(scaler_predict)

#print(predict)

[[0.11023994]
 [3.9487932 ]
 [7.0969977 ]
 ...
 [6.832763  ]
 [3.345881  ]
 [6.595546  ]]


In [70]:
print(tf.nn.relu(predict, name = 'relu'))

tf.Tensor(
[[0.11023994]
 [3.9487932 ]
 [7.0969977 ]
 ...
 [6.832763  ]
 [3.345881  ]
 [6.595546  ]], shape=(3263, 1), dtype=float32)


In [80]:
print(np.maximum(predict[0],0))

[0.11023994]


In [100]:
def ReLU(x):

    if np.maximum(0,x) == 0.00:
        return 0
    else:
        return 1
    #return (np.maximum(0,x))

predict_relu = []

for i in predict:
    predict_relu.append(ReLU(i))
    

test_target = pd.Series(predict_relu)

print(test_target)


0       1
1       1
2       1
3       1
4       1
       ..
3258    1
3259    1
3260    1
3261    1
3262    1
Length: 3263, dtype: int64


In [101]:
print(df_test['text'])

0                      Just happened a terrible car crash
1       Heard about #earthquake is different cities, s...
2       there is a forest fire at spot pond, geese are...
3                Apocalypse lighting. #Spokane #wildfires
4           Typhoon Soudelor kills 28 in China and Taiwan
                              ...                        
3258    EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259    Storm in RI worse than last hurricane. My city...
3260    Green Line derailment in Chicago http://t.co/U...
3261    MEG issues Hazardous Weather Outlook (HWO) htt...
3262    #CityofCalgary has activated its Municipal Eme...
Name: text, Length: 3263, dtype: object


In [103]:
df_test['target'] = test_target

In [104]:
print(df_test)

         id keyword location  \
0         0     NaN      NaN   
1         2     NaN      NaN   
2         3     NaN      NaN   
3         9     NaN      NaN   
4        11     NaN      NaN   
...     ...     ...      ...   
3258  10861     NaN      NaN   
3259  10865     NaN      NaN   
3260  10868     NaN      NaN   
3261  10874     NaN      NaN   
3262  10875     NaN      NaN   

                                                   text  target  
0                    Just happened a terrible car crash       1  
1     Heard about #earthquake is different cities, s...       1  
2     there is a forest fire at spot pond, geese are...       1  
3              Apocalypse lighting. #Spokane #wildfires       1  
4         Typhoon Soudelor kills 28 in China and Taiwan       1  
...                                                 ...     ...  
3258  EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...       1  
3259  Storm in RI worse than last hurricane. My city...       1  
3260  Green Line dera

In [109]:
df_test[['id','target']].to_csv('submission.csv',index = False)